# Modelo de Identificação de Fraudes

- Informações da base:
    - 'Time' (tempo): contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. 
    - 'Amount' (valor): é o valor da transação 
    - 'Class' (classe): é a variável de resposta e assume valor 1 em caso de fraude e 0 caso contrário.
    - "Infelizmente, devido a questões de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Características V1, V2, … V28 são os principais componentes obtidos com PCA"

In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude
C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude


In [3]:
# Importações necessárias 

import sys
import os
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import seaborn as sns
import warnings

from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.model_selection import (
    StratifiedKFold, 
    KFold,
    cross_validate,
)

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    make_scorer,
    precision_recall_curve,
)

from src.config.paths import DADOS_CREDICARD, DADOS_CREDICARD_TRATADO
from src.config.paths import MODELS_DIR

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [4]:
# Importando a base

transacoes = pd.read_parquet(DADOS_CREDICARD_TRATADO)

In [4]:
# Embaralha as transações para garantir aleatoriedade

transacoes = transacoes.sample(frac=1, random_state=42)  

### Pré-processamento: 

- Utilizaremos o **PowerTransformer** devido ao grande número de outliers, **exceto em 'Time'**.<br>
- Em **'Time'** utilizaremos o **MinMaxScaler**.

In [5]:
# Criando os escaladores

power_transformer = PowerTransformer(method='yeo-johnson')  # Funciona para valores positivos e negativos
minmax_scaler = MinMaxScaler()

# Aplicando a normalização nas colunas específicas

transacoes['Amount'] = power_transformer.fit_transform(transacoes[['Amount']])
transacoes['Time'] = minmax_scaler.fit_transform(transacoes[['Time']])

In [6]:
# Separando X e y

X = transacoes.drop(columns='Class')
y = transacoes['Class']

### XGBoost

In [7]:
# KFold com 10 dobras

kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Variáveis para armazenar as métricas de cada dobra

accuracy_scores = []
precision_scores = []
recall_scores = []

In [8]:
# Para iterar pelas dobras

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Treinando o modelo XGBoost
    
    model = xgb.XGBClassifier(
        max_depth=9,
        n_estimators=200,
        learning_rate=0.3,
        random_state=0,
        scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
    )
    
    model.fit(X_train, y_train)
    
    # Fazendo previsões
    
    y_pred = model.predict(X_test)
    
    # Calculando as métricas
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, zero_division=0))
    recall_scores.append(recall_score(y_test, y_pred, zero_division=0))

In [9]:
# Calculando a média das métricas

mean_accuracy = np.mean(accuracy_scores)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)

In [10]:
# Exibindo os resultados médios

print("Média das métricas após 10 dobras:")
print(f"Acurácia: {mean_accuracy:.4f}")
print(f"Precisão: {mean_precision:.4f}")
print(f"Recall: {mean_recall:.4f}")

Média das métricas após 10 dobras:
Acurácia: 0.9996
Precisão: 0.9227
Recall: 0.8102


### Salvando o modelo para testes com novos dados. 

**Simulando modelo em produção real.**

In [11]:
# Salvaando o melhor modelo em formato .pkl na pasta 'models'

# Caminho completo do arquivo

caminho_modelo = MODELS_DIR / 'best_model_xgboost.pkl'

# Salvando o modelo

joblib.dump(model, caminho_modelo)
print(f"Modelo salvo em: {caminho_modelo}")

Modelo salvo em: C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude\models\best_model_xgboost.pkl


## Análise Final do Modelo de Detecção de Fraudes

<br>O modelo de detecção de fraudes foi avaliado utilizando validação cruzada com 10 dobras, resultando nas seguintes métricas médias:<br>

- #### Acurácia: 99,96%
- #### Precisão: 92,27%
- #### Recall: 81,02% 

<br>Dado que a empresa aplicará esse modelo para transações corriqueiras com um valor máximo de **R$5.000,00**, a principal preocupação é minimizar falsos positivos para evitar reclamações de clientes.<br><br>

#### Interpretação das Métricas:<br>

- #### Alta Precisão (92,27%):

Isso indica que a maioria das transações identificadas como fraude realmente são fraudes, o que significa que poucos clientes legítimos terão suas transações bloqueadas indevidamente.
Esse é um ponto positivo, pois reduz o impacto negativo sobre a experiência do usuário.<br><br>

- #### Recall Moderado (81,02%):

O modelo consegue capturar 81% das fraudes reais, o que é bom, mas há um 19% de fraudes não detectadas.
Como o foco principal é reduzir falsos positivos, esse recall é aceitável dentro do contexto do negócio.<br><br>

- #### Alta Acurácia (99,96%):

Esse valor é muito alto, mas deve ser interpretado com cuidado, pois fraudes são eventos raros.
Como o dataset é desbalanceado, a acurácia pode estar sendo inflada pelo grande número de transações legítimas corretamente classificadas.